In [1]:
!pip3 install -qq -U wandb

2023-05-22 21:47:35.345102: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 21:47:36.336113: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-22 21:47:36.336244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
import sys
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow import keras as k
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras import backend
from keras.engine import base_layer
from keras.engine import base_preprocessing_layer
from keras.engine import input_spec
from keras.layers.preprocessing import preprocessing_utils as utils
from keras.utils import image_utils
from keras.utils import conv_utils
from matplotlib import pyplot
from tqdm.keras import TqdmCallback
import wandb
from wandb.keras import WandbMetricsLogger

2023-05-24 13:11:16.604926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 13:11:17.620743: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-24 13:11:17.620875: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [3]:
# load train and test dataset
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    # one hot encode target values
    trainY = k.utils.to_categorical(trainY)
    testY = k.utils.to_categorical(testY)
    return trainX, trainY, testX, testY

In [4]:
# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

In [5]:
# scale pixel : normalize and center pixel
def prep_pixels_mean_std(train, test):
 # convert from integers to floats
 train_norm = train.astype('float32')
 test_norm = test.astype('float32')
 # normalize to range 0-1
 train_norm = (train_norm - train_norm.mean())/train_norm.std()
 test_norm = (test_norm - test_norm.mean())/test_norm.std()
 # return normalized images
 return train_norm, test_norm

# Nouvelle section

In [6]:
def summarize_diagnostics(history):
    min_loss = min(history.history['val_loss'])
    print(f'Minimum validation loss {min_loss} at epoch {history.history["val_loss"].index(min_loss)}')
    best_acc = max(history.history['val_accuracy'])
    print(f'Best validation accuracy {best_acc} at epoch {history.history["val_accuracy"].index(best_acc)}')
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.tight_layout()
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()
    with open('stats.txt', 'w') as f:
        f.write(f'{min_loss} @ {history.history["val_loss"].index(min_loss)}\n{best_acc} @ {history.history["val_accuracy"].index(best_acc)}\n')

In [7]:
H_AXIS = -3
W_AXIS = -2


def convert_inputs(inputs, dtype=None):
    if isinstance(inputs, dict):
        raise ValueError(
            "This layer can only process a tensor representing an image or "
            f"a batch of images. Received: type(inputs)={type(inputs)}."
            "If you need to pass a dict containing "
            "images, labels, and bounding boxes, you should "
            "instead use the preprocessing and augmentation layers "
            "from `keras_cv.layers`. See docs at "
            "https://keras.io/api/keras_cv/layers/"
        )
    inputs = utils.ensure_tensor(inputs, dtype=dtype)
    return inputs


class RandomCrop(base_layer.BaseRandomLayer):

    def __init__(self, height, width, seed=None, **kwargs):
        base_preprocessing_layer.keras_kpl_gauge.get_cell("RandomCrop").set(
            True
        )
        super().__init__(
            **kwargs, autocast=False, seed=seed, force_generator=True
        )
        self.height = height
        self.width = width
        self.seed = seed

    def call(self, inputs, training=True):
        inputs = convert_inputs(inputs, dtype=self.compute_dtype)
        input_shape = tf.shape(inputs)
        h_diff = input_shape[H_AXIS] - self.height
        w_diff = input_shape[W_AXIS] - self.width

        def random_crop():
            dtype = input_shape.dtype
            rands = self._random_generator.random_uniform(
                [2], 0, dtype.max, dtype
            )
            h_start = rands[0] % (h_diff + 1)
            w_start = rands[1] % (w_diff + 1)
            return tf.image.crop_to_bounding_box(
                inputs, h_start, w_start, self.height, self.width
            )

        def resize():
            outputs = image_utils.smart_resize(
                inputs, [self.height, self.width]
            )
            # smart_resize will always output float32, so we need to re-cast.
            return tf.cast(outputs, self.compute_dtype)

        return tf.cond(
            tf.reduce_all((training, h_diff >= 0, w_diff >= 0)),
            random_crop,
            resize,
        )

    def compute_output_shape(self, input_shape):
        input_shape = tf.TensorShape(input_shape).as_list()
        input_shape[H_AXIS] = self.height
        input_shape[W_AXIS] = self.width
        return tf.TensorShape(input_shape)

    def get_config(self):
        config = {
            "height": self.height,
            "width": self.width,
            "seed": self.seed,
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [8]:
class ZeroPadding2D(base_layer.Layer):

    def __init__(self, padding=(1, 1), data_format=None, **kwargs):
        super().__init__(**kwargs)
        self.data_format = conv_utils.normalize_data_format(data_format)
        if isinstance(padding, int):
            self.padding = ((padding, padding), (padding, padding))
        elif hasattr(padding, "__len__"):
            if len(padding) != 2:
                raise ValueError(
                    f"`padding` should have two elements. Received: {padding}."
                )
            height_padding = conv_utils.normalize_tuple(
                padding[0], 2, "1st entry of padding", allow_zero=True
            )
            width_padding = conv_utils.normalize_tuple(
                padding[1], 2, "2nd entry of padding", allow_zero=True
            )
            self.padding = (height_padding, width_padding)
        else:
            raise ValueError(
                "`padding` should be either an int, "
                "a tuple of 2 ints "
                "(symmetric_height_pad, symmetric_width_pad), "
                "or a tuple of 2 tuples of 2 ints "
                "((top_pad, bottom_pad), (left_pad, right_pad)). "
                f"Received: {padding}."
            )
        self.input_spec = input_spec.InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        input_shape = tf.TensorShape(input_shape).as_list()
        if self.data_format == "channels_first":
            if input_shape[2] is not None:
                rows = input_shape[2] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None
            if input_shape[3] is not None:
                cols = input_shape[3] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None
            return tf.TensorShape([input_shape[0], input_shape[1], rows, cols])
        elif self.data_format == "channels_last":
            if input_shape[1] is not None:
                rows = input_shape[1] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None
            if input_shape[2] is not None:
                cols = input_shape[2] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None
            return tf.TensorShape([input_shape[0], rows, cols, input_shape[3]])

    def call(self, inputs, training=None):
        if training:
            return backend.spatial_2d_padding(
                inputs, padding=self.padding, data_format=self.data_format
            )
        return inputs

    def get_config(self):
        config = {"padding": self.padding, "data_format": self.data_format}
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [9]:
def define_resnet(wdecay=0):
    inputs = k.Input(shape=(32, 32, 3))
    
    #DATA AUGMENTATION (inactive during inference)
    aug = ZeroPadding2D(4)(inputs)
    aug = k.layers.RandomFlip(mode='horizontal')(aug)
    aug = RandomCrop(32, 32)(aug)
    
    #layer1 = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(aug)
    layer1 = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(aug)
    layer1 = k.layers.BatchNormalization()(layer1)
    layer1 = k.layers.Activation(k.activations.relu)(layer1)
    #FIRST STACK
    #x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(layer1)
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(layer1)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    #x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([layer1, x]))
    #x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    #x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    #x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    #x = k.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    #SECOND STACK
    #x = k.layers.Conv2D(32, (3, 3), strides=(2, 2), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.Conv2D(32, (3, 3), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Conv2D(32, (1, 1), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    #x = k.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    #x = k.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    #x = k.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    #x = k.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    #THIRD STACK
    #x = k.layers.Conv2D(64, (3, 3), strides=(2, 2), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.Conv2D(64, (3, 3), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    #x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Conv2D(64, (1, 1), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    #x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    #x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    #x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    #x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
    #    kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    res = k.layers.GlobalAveragePooling2D()(res)
    
    outputs = k.layers.Dense(10, activation='softmax')(res)
    
    return k.Model(inputs=inputs, outputs=outputs, name='ResNet_20')

In [10]:
resnet = define_resnet()
resnet.summary()

2023-05-24 13:11:46.109886: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-24 13:11:46.280110: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-24 13:11:46.280373: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-24 13:11:46.282674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "ResNet_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 32, 32, 3)   0           ['input_1[0][0]']                
                                                                                                  
 random_flip (RandomFlip)       (None, 32, 32, 3)    0           ['zero_padding2d[0][0]']         
                                                                                       

In [12]:
def define_resnetplus(wdecay=0):
    inputs = k.Input(shape=(32, 32, 3))
    
    #DATA AUGMENTATION (inactive during inference)
    aug = ZeroPadding2D(4)(inputs)
    #aug = k.layers.RandomFlip(mode='horizontal')(aug)
    aug = RandomCrop(32, 32)(aug)
    
    layer1 = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(aug)
    layer1 = k.layers.BatchNormalization()(layer1)
    layer1 = k.layers.Activation(k.activations.relu)(layer1)
    #FIRST STACK
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(layer1)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([layer1, x]))
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(16, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    downres = k.layers.Conv2D(16, (1, 1), strides=(2, 2), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    
    #SECOND STACK
    x = k.layers.Conv2D(32, (3, 3), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(downres)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Conv2D(32, (1, 1), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(32, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    downres = k.layers.Conv2D(32, (1, 1), strides=(2, 2), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    
    #THIRD STACK
    x = k.layers.Conv2D(64, (3, 3), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(downres)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Conv2D(64, (1, 1), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Activation(k.activations.relu)(x)
    x = k.layers.Conv2D(64, (3, 3), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    #x = k.layers.Activation(k.activations.relu)(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    res = k.layers.GlobalAveragePooling2D()(res)
    
    outputs = k.layers.Dense(10, activation='softmax')(res)
    
    return k.Model(inputs=inputs, outputs=outputs, name='ResNet_20')

In [13]:
#resnetplus = define_resnetplus()
#resnetplus.summary()

In [14]:
def define_resnext(wdecay=0):
    inputs = k.Input(shape=(32, 32, 3))
    
    #DATA AUGMENTATION (inactive during inference)
    aug = ZeroPadding2D(4)(inputs)
    #aug = k.layers.RandomFlip(mode='horizontal')(aug)
    aug = RandomCrop(32, 32)(aug)
    
    layer1 = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(aug)
    layer1 = k.layers.BatchNormalization()(layer1)
    
    #FIRST STACK
    x = k.layers.Conv2D(64, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(layer1)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Conv2D(256, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(layer1)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(64, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(64, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    #SECOND STACK
    x = k.layers.Conv2D(128, (1, 1), strides=(2, 2), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(512, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Conv2D(512, (1, 1), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(128, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(512, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(128, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(512, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    #THIRD STACK
    x = k.layers.Conv2D(256, (1, 1), strides=(2, 2), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(1024, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Conv2D(1024, (1, 1), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(256, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(1024, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.Conv2D(256, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(1024, (1, 1), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    res = k.layers.GlobalAveragePooling2D()(res)
    
    outputs = k.layers.Dense(10, activation='softmax')(res)
    
    return k.Model(inputs=inputs, outputs=outputs, name='ResNext_29')

In [15]:
resnext = define_resnext()
resnext.summary()

Model: "ResNext_29"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d_8 (ZeroPadding2  (None, 32, 32, 3)   0           ['input_9[0][0]']                
 D)                                                                                               
                                                                                                  
 random_crop_8 (RandomCrop)     (None, 32, 32, 3)    0           ['zero_padding2d_8[0][0]']       
                                                                                                  
 conv2d_168 (Conv2D)            (None, 32, 32, 64)   1792        ['random_crop_8[0][0]'] 

                                                                                                  
 batch_normalization_163 (Batch  (None, 16, 16, 128)  512        ['conv2d_180[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv2d_181 (Conv2D)            (None, 16, 16, 512)  66048       ['batch_normalization_163[0][0]']
                                                                                                  
 conv2d_182 (Conv2D)            (None, 16, 16, 512)  131584      ['activation_226[0][0]']         
                                                                                                  
 batch_normalization_164 (Batch  (None, 16, 16, 512)  2048       ['conv2d_181[0][0]']             
 Normalization)                                                                                   
          

 conv2d_194 (Conv2D)            (None, 8, 8, 256)    590080      ['batch_normalization_174[0][0]']
                                                                                                  
 batch_normalization_175 (Batch  (None, 8, 8, 256)   1024        ['conv2d_194[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv2d_195 (Conv2D)            (None, 8, 8, 1024)   263168      ['batch_normalization_175[0][0]']
                                                                                                  
 batch_normalization_176 (Batch  (None, 8, 8, 1024)  4096        ['conv2d_195[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_79 (A

In [16]:
def define_convnext(wdecay=0):
    inputs = k.Input(shape=(32, 32, 3))
    
    #DATA AUGMENTATION (inactive during inference)
    aug = ZeroPadding2D(4)(inputs)
    #aug = k.layers.RandomFlip(mode='horizontal')(aug)
    aug = RandomCrop(32, 32)(aug)
    
    layer1 = k.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(aug)
    #layer1 = k.layers.BatchNormalization()(layer1)
    #FIRST STACK
    x = k.layers.DepthwiseConv2D((3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(layer1)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(64, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([layer1, x]))
    x = k.layers.DepthwiseConv2D((3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(64, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.DepthwiseConv2D((3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(64, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    #SECOND STACK
    x = k.layers.DepthwiseConv2D((3, 3), strides=(2, 2), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(512, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(128, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    res = k.layers.Conv2D(128, (1, 1), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.DepthwiseConv2D((3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(512, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(128, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.DepthwiseConv2D((3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(512, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(128, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    #THIRD STACK
    x = k.layers.DepthwiseConv2D((3, 3), strides=(2, 2), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(1024, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    res = k.layers.Conv2D(256, (1, 1), strides=(2, 2), kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.DepthwiseConv2D((3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(1024, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    x = k.layers.DepthwiseConv2D((3, 3), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(res)
    x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(1024, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    x = k.layers.Conv2D(256, (1, 1), activation='relu', kernel_initializer='he_uniform',
        kernel_regularizer=k.regularizers.l2(wdecay), padding='same')(x)
    #x = k.layers.BatchNormalization()(x)
    res = k.layers.Activation(k.activations.relu)(k.layers.Add()([res, x]))
    
    res = k.layers.GlobalAveragePooling2D()(res)
    
    outputs = k.layers.Dense(10, activation='softmax')(res)
    
    return k.Model(inputs=inputs, outputs=outputs, name='ConvNext')

In [17]:
convnext = define_convnext()
convnext.summary()

Model: "ConvNext"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 32, 32, 3)   0           ['input_3[0][0]']                
 D)                                                                                               
                                                                                                  
 random_crop_2 (RandomCrop)     (None, 32, 32, 3)    0           ['zero_padding2d_2[0][0]']       
                                                                                                  
 conv2d_54 (Conv2D)             (None, 32, 32, 64)   1792        ['random_crop_2[0][0]']   

                                                                                                  
 add_22 (Add)                   (None, 16, 16, 128)  0           ['activation_21[0][0]',          
                                                                  'conv2d_65[0][0]']              
                                                                                                  
 activation_22 (Activation)     (None, 16, 16, 128)  0           ['add_22[0][0]']                 
                                                                                                  
 depthwise_conv2d_5 (DepthwiseC  (None, 16, 16, 128)  1280       ['activation_22[0][0]']          
 onv2D)                                                                                           
                                                                                                  
 batch_normalization_52 (BatchN  (None, 16, 16, 128)  512        ['depthwise_conv2d_5[0][0]']     
 ormalizat

In [11]:
def resnet_scheduler(epoch, lr):
    #if epoch == 75:
    #    lr *= 0.2
    #if epoch == 90:
    #    lr *= 0.1
    #if epoch == 100:
    #    lr *= 0.1
    #if epoch == 135:
    #    lr *= 0.1
    #if epoch == 150:
    #    lr *= 0.1
    #if epoch == 150:
    #    lr *= 0.2
    #if epoch == 175:
    #    lr *= 0.5
    #if epoch == 225:
    #    lr *= 0.1
    if epoch != 0 and epoch % 5 == 0:
    #   return lr * 0.87
        return lr * 0.95
    return lr

In [12]:
def resnet_restart(epoch, lr):
  if epoch < 90:
    eta_max = 0.1
    eta_max = 0.1
  elif epoch >= 90 and epoch < 135:
    eta_max = 0.05
    eta_min = 5e-5
  else:
    eta_max = 0.01
    eta_min = 1e-5
  epoch_period = 25
  lr_t = eta_min + 0.5 * (eta_max - eta_min) * (1 + np.cos(np.pi * (epoch % epoch_period) / epoch_period))
  return lr_t

In [24]:
def resnet_warm_up(epoch, lr):
  eta_max = 0.05
  eta_min = 1e-7
  epoch_period = 180

  if epoch < 20:
    return eta_max * epoch / 20
  else:
    return eta_min + 0.5 * (eta_max - eta_min) * (1 + np.cos(np.pi * (epoch - 20) / epoch_period))

In [28]:
def prep_pixels_mean(train, test, valid):
    train -= train.mean()
    test -= train.mean()
    valid -= train.mean()
    return train, test, valid

In [16]:
def run_test(ep=180, learning_rate=0.1, wdecay=0, optimizer='SGD', run=True):
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels_mean(trainX, testX)
    # pad image
    trainX = padding(trainX)
    # define model
    model = define_resnet(wdecay)
    if optimizer == 'SGD':
        opt = k.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    elif optimizer == 'AdamW':
        opt = k.optimizers.experimental.AdamW()
    model.compile(optimizer=opt, loss=k.losses.CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
    if optimizer == 'SGD':
        lr_sched = k.callbacks.LearningRateScheduler(resnet_restart)
    if run:
        history = model.fit(trainX, trainY, epochs=ep, batch_size=128, validation_split=0.1, verbose=0,
            callbacks=[lr_sched, TqdmCallback(verbose=1)])
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        # learning curves
        summarize_diagnostics(history)

In [17]:
def mixup(X, Y, mixup_alpha):
    #Beta samples
    g1 = tf.random.gamma(shape=[X.shape[0]], alpha=mixup_alpha)
    g2 = tf.random.gamma(shape=[X.shape[0]], alpha=mixup_alpha)
    beta = (g1 / (g1 + g2))
    #Mixup
    lambdaX = tf.reshape(beta, (X.shape[0], 1, 1, 1))
    lambdaY = tf.reshape(beta, (Y.shape[0], 1))
    indices = tf.range(start=0, limit=X.shape[0], dtype=tf.int32)
    shuffle = tf.random.shuffle(indices)
    X_shuffled = tf.gather(X, shuffle)
    Y_shuffled = tf.gather(Y, shuffle)
    
    mixedX = lambdaX * X + (1-lambdaX) * X_shuffled
    mixedY = lambdaY * Y + (1-lambdaY) * Y_shuffled
    
    return mixedX, mixedY

In [18]:
def cutmix(X, Y, cutmix_alpha):
    print('Applying cutmix data augmentation...')
    mixedX = np.zeros((X.shape[0], X.shape[1], X.shape[2], X.shape[3]))
    mixedY = np.zeros((Y.shape[0], Y.shape[1]))
    indices = tf.range(start=0, limit=X.shape[0], dtype=tf.int32)
    shuffle = tf.random.shuffle(indices)
    X_shuffled = tf.gather(X, shuffle)
    Y_shuffled = tf.gather(Y, shuffle)
    
    for i in range(X.shape[0]):
        #Beta samples
        g1 = tf.random.gamma(shape=[1], alpha=cutmix_alpha)
        g2 = tf.random.gamma(shape=[1], alpha=cutmix_alpha)
        beta = (g1 / (g1 + g2))[0]

        rx = tf.random.uniform((1,), minval=0, maxval=X.shape[1], dtype=tf.int32)[0]
        ry = tf.random.uniform((1,), minval=0, maxval=X.shape[2], dtype=tf.int32)[0]
        rw = X.shape[2] * tf.math.sqrt(1.0 - beta)
        rh = X.shape[1] * tf.math.sqrt(1.0 - beta)
        rw = tf.cast(rw, tf.int32)
        rh = tf.cast(rh, tf.int32)
        bx = tf.clip_by_value(rx - rw//2, 0, X.shape[1])
        by = tf.clip_by_value(ry - rh//2, 0, X.shape[2])
        rw = tf.clip_by_value(rx + rw//2, 0, X.shape[1])
        rh = tf.clip_by_value(ry + rw//2, 0, X.shape[2])
        rw -= bx
        rh -= by
        rw = rw if rw > 0 else 1
        rh = rh if rh > 0 else 1
        
        image1 = np.array(X[i])
        image2 = np.array(X_shuffled[i])
        crop2 = tf.image.crop_to_bounding_box(image2, bx, by, rw, rh)
        image2 = tf.image.pad_to_bounding_box(crop2, bx, by, X.shape[1], X.shape[2])
        crop1 = tf.image.crop_to_bounding_box(image1, bx, by, rw, rh)
        padded1 = tf.image.pad_to_bounding_box(crop1, bx, by, X.shape[1], X.shape[2])
        image1 -= padded1
        mixedX[i] = image1 + image2
        
        lbda = 1 - (rw * rh) / (X.shape[1] * X.shape[2])
        lbda = tf.cast(lbda, tf.float32)
        mixedY[i] = lbda * Y[i] + (1 - lbda) * Y_shuffled[i]
    
    print('cutmix completed.')
    return mixedX, mixedY

In [25]:
def run_test_next(
    ep=180, normalize=True, learning_rate=0.1, wdecay=0, smoothing=0,
    optimizer='SGD', mixup_alpha=0, cutmix_alpha=0, run=True):
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    trainX = trainX.astype('float32')
    testX = testX.astype('float32')
    store = []
    store.append(np.array(trainX[0]))
    store.append(np.array(trainX[1]))
    store.append(np.array(trainX[2]))
    # prepare pixel data

    store.append(np.array(trainX[0]))
    store.append(np.array(trainX[1]))
    store.append(np.array(trainX[2]))

    np.random.seed(33)
    v_indices = np.random.choice(range(50000), size=(5000), replace=False)
    validX = trainX[v_indices]
    validY = trainY[v_indices]
    trainX = np.delete(trainX, v_indices, axis=0)
    trainY = np.delete(trainY, v_indices, axis=0)
    
    if normalize:
        trainX, testX, validX = prep_pixels_mean(trainX, testX, validX)
    else:
        trainX /= 255.0
        testX /= 255.0
        validX /= 255.0
    
    # apply mixup (typical is 0.2)
    if mixup_alpha > 0:
        trainX, trainY = mixup(trainX, trainY, mixup_alpha)
    # apply cutmix (typical is 0.2) CAUTION: takes ~20 minutes
    if cutmix_alpha > 0:
        trainX, trainY = cutmix(trainX, trainY, cutmix_alpha)
    store.append(np.array(trainX[0]))
    store.append(np.array(trainX[1]))
    store.append(np.array(trainX[2]))
    
    #for i in range(9):
    #    pyplot.subplot(330 + 1 + i)
    #    pyplot.imshow(store[i])
    #pyplot.show()

    # define model
    model = define_resnet(wdecay)
    wandb.init()
    if optimizer == 'SGD':
        opt = k.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    elif optimizer == 'Adam':
        opt = k.optimizers.Adam(learning_rate=learning_rate / 10)
    elif optimizer == 'AdamW':
        opt = k.optimizers.experimental.AdamW(learning_rate=learning_rate / 10)
    model.compile(optimizer=opt,
        loss=k.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=smoothing),
        metrics=['accuracy'])
    lr_sched = k.callbacks.LearningRateScheduler(resnet_warm_up)
    if run:
        history = model.fit(trainX, trainY, epochs=ep, batch_size=128, validation_data=(validX, validY), verbose=0,
            callbacks=[WandbMetricsLogger(), lr_sched, TqdmCallback(verbose=1)])
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        # learning curves
        summarize_diagnostics(history)

In [ ]:
run_test_next(ep=180, normalize=True, learning_rate=0.05, wdecay=0.0005)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]